In [5]:
function nonLinearFD(a, b, alpha, beta, N, TOL, max, f, fy, fyy)
    w = zeros(N)
    _a = zeros(N)
    _b = zeros(N)
    _c = zeros(N)
    _d = zeros(N)
    
    h = (b - a) / (N + 1)
    
    for i = 1:N
        w[i] = alpha + i*((beta - alpha)/(b - a))*h
    end

    k = 1
    while k <= max
        x = a + h
        t = (w[2] - alpha) / (2*h)
        _a[1] = 2 + (h^2)*fy(x, w[1], t)
        _b[1] = -1 + (h/2)*fyy(x, w[1], t)
        _d[1] = -(2*w[1] - w[2] - alpha + (h^2)*f(x, w[1], t))
        for i = 2:N-1
            x = a + i*h
            t = (w[i+1] - w[i-1]) / (2*h)
            _a[i] = 2 + (h^2)*fy(x, w[i], t)
            _b[i] = -1 + (h/2)*fyy(x, w[i], t)
            _c[i] = -1 - (h/2)*fyy(x, w[i], t)
            _d[i] = -(2*w[i] - w[i+1] - w[i-1] + (h^2)*f(x, w[i], t))
        end
        x = b - h
        t = (beta - w[N-1]) / (2*h)
        _a[N] = 2 + h^2*fy(x, w[N], t)
        _c[N] = -1 - (h/2)*fyy(x, w[N], t)
        _d[N] = -(2*w[N] - w[N-1] - beta + (h^2)*f(x, w[N], t))
        

        ################# tridiag #################
        l = zeros(N)
        v = zeros(N)
        u = zeros(N)
        z = zeros(N)
        l[1] = _a[1]
        u[1] = _b[1] / _a[1]
        z[1] = _d[1] / l[1]
        for i = 2:N-1
            l[i] = _a[i] - _c[i]*u[i-1]
            u[i] = _b[i] / l[i]
            z[i] = (_d[i] - _c[i]*z[i-1])/l[i]
        end
        l[N] = _a[N] - _c[N]*u[N-1]
        z[N] = (_d[N] - _c[N]*z[N-1]) / l[N]
        v[N] = z[N]
        w[N] += v[N]
        for i = N-1:-1:1
            v[i] = z[i] - u[i]*v[i+1]
            w[i] += v[i]
        end
        ##########################################


        if norm(v) <= TOL
            # add boundaries
            _w = zeros(N+2)
            x = zeros(N+2)
            _w[1] = alpha
            _w[N+2] = beta
            x[1] = a
            x[N+2] = b
            for i = 1:N
                _w[i+1]= w[i]
                x[i] = a + i*h
            end
            #w = _w
            return x, _w
        end
        k += 1
    end
    println("maximum iterations")
end

a = 1 # endpoint
b = 3 # endpoint
alpha = 17 # boundary condition
beta = 43/3 # boundary condition
N = 19 # number of chunks
TOL = 1e-8 # tolerance
max = 100000 # maximum iterations
f = (x, w, t) -> (1/8)*(32 + 2*x^3 - w*t) # f = y''
fy = (x, w, t) -> (-1/8)*t # fy - t = y' w = y
fyy = (x, w, t) -> (-1/8)*w*f(x, w, t) # fy' - t = y' w = y
println(nonLinearFD(a, b, alpha, beta, N, TOL, max, f, fy, fyy))

([1.1,1.2,1.3,1.4,1.5,1.6,1.7000000000000002,1.8,1.9,2.0,2.1,2.2,2.3,2.4000000000000004,2.5,2.6,2.7,2.8,2.9000000000000004,0.0,3.0],[17.0,15.754502533520938,14.771739639124213,13.995677413709755,13.386296535884732,12.91425238777826,12.557538206123926,12.299326264930219,12.12652885329212,12.028813799359055,11.997915414732235,12.027142365680815,12.111019800497383,12.245024864745664,12.425388361064465,12.648944029200072,12.913012622282949,13.2153117552817,13.553885079919342,13.927046118445327,14.333333333333334])
